In [17]:
#movie rec using collaborative data and matrix factorization
import csv
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from scipy import optimize
from sklearn.decomposition import TruncatedSVD

#some code from http://alexabate.github.io/2016/11/05/movie-lens.html
#some code from https://nikhilwins.wordpress.com/2015/09/18/movie-recommendations-how-does-netflix-do-it-a-9-step-coding-intuitive-guide-into-collaborative-filtering/

user_item_ratings = pd.read_csv('ratings.csv', sep=',')
#user_item_ratings.head()
number_of_unique_users = len(user_item_ratings['userId'].unique())
number_of_unique_movies = len(user_item_ratings['movieId'].unique())
number_of_ratings = len(user_item_ratings)
#print user_item_ratings
print number_of_unique_users
print number_of_unique_movies

ratingsMatrix=pd.pivot_table(user_item_ratings, values='rating', fill_value=0, columns='userId', index='movieId')#
#print ratingsMatrix.head

ratingsMatrix2=ratingsMatrix.iloc[:5000,:]

print ratingsMatrix2.shape

did_rate = (ratingsMatrix2 != 0) * 1


#n_movies=number_of_unique_movies
n_movies=ratingsMatrix2.shape[0]
n_users=number_of_unique_users


#normalize_ratings from https://nikhilwins.wordpress.com/2015/09/18/movie-recommendations-how-does-netflix-do-it-a-9-step-coding-intuitive-guide-into-collaborative-filtering/
def normalize_ratings(ratings, did_rate):
    num_movies = ratings.shape[0]
    ratings_mean = np.zeros(shape = (num_movies, 1))
    ratings_norm = np.zeros(shape = ratings.shape)
    
    for i in range(0, num_movies):
        # Get all the indexes where there is a 1
        idx = np.where(did_rate.iloc[i,:] ==1)[0]
        # Calculate mean rating of ith movie only from users that gave a rating
        ratings_mean[i] = np.mean(ratings.iloc[i, idx])
         #ratings_mean[i]
        ratings_norm[i, idx] = ratings.iloc[i, idx] - ratings_mean[i]
        #(EKC: make sure these indices aren't off by one)
        #print i
    return (ratings_norm, ratings_mean)

ratings_norm, ratings_mean = normalize_ratings(ratingsMatrix2, did_rate)

did_rate=did_rate.as_matrix()

svd = TruncatedSVD(n_components=3, n_iter=10, random_state=42)
X_tr = svd.fit_transform(ratings_norm)
print X_tr.shape

X_o = svd.inverse_transform(X_tr)
#that would be the predicted ratings
difference = X_o*did_rate-ratings_norm
cost = np.sum((difference)**2)/2
print cost

671
9066
(5000, 671)
(5000, 3)
29637.9322878


In [18]:
ratings_test=ratingsMatrix.iloc[5000:,:]
print ratings_test.shape
did_rate_test= (ratings_test != 0) * 1
ratings_test_norm, test_mean=normalize_ratings(ratings_test, did_rate_test)
did_rate_test=did_rate_test.as_matrix()

print ratings_test_norm.shape
X_t_test=svd.transform(ratings_test_norm)
#this transforms the ratings_test_norm matrix using the model fit to ratings_norm
X_o_test = svd.inverse_transform(X_t_test)
#that would be the predicted ratings
difference_test = X_o_test*did_rate_test-ratings_test_norm
cost_test = np.sum((difference_test)**2)/2
print cost_test



(4066, 671)
(4066, 671)
5611.34021819


In [24]:


# get back the latent features using matrix multplication? (i have both X and its transformation)
# X_o=X_tr*Beta (Beta=theta, I don't know why I use Beta instead) 
# (there must be a way to access from svd using sklearn though)
Beta=X_tr.T.dot(X_o)
print Beta.shape
# (this describes each user's weights, or inferred preference, for each feature)

# and that should be equal to what you get using the transformed (and inverse-transformed) test data
# if the model is the same. check:
Beta_test=X_t_test.T.dot(X_o_test)
print np.sum(Beta-Beta_test)
#(it's basically zero, just some rounding/precision error accumulated)

(3, 671)
-7.83757059253e-10


In [ ]:
# can do cross-validation using different random splits of the data to train and test the model 
# train and test different models on the SAME splits for most direct model comparison
# (e.g., to see whether adding features is helpful)